In [1948]:
import requests
from bs4 import BeautifulSoup
from fuzzywuzzy import fuzz

from selenium import webdriver
from selenium.webdriver.common.by import By
from operator import itemgetter

In [1949]:
#funcion para consultar en la pagina de farmacity
def obtener_info_farmacity(nombre_producto, umbral_similitud=60):
    # Inicializa el navegador Chrome
    driver = webdriver.Chrome()

    # Abre la URL de Farmacity usando un f-string para insertar el nombre del producto
    url = f"https://www.farmacity.com/{nombre_producto}?_q={nombre_producto}&map=ft"
    driver.get(url)
    
    # Encuentra los elementos que contienen el título de los productos
    titles = driver.find_elements(By.XPATH, '//span[@class="vtex-product-summary-2-x-productBrand vtex-product-summary-2-x-brandName t-body"]')

    # Encuentra los elementos que contienen el precio de los productos
    prices = driver.find_elements(By.XPATH, '//span[@class="vtex-product-price-1-x-currencyContainer"]')
       
    # Crear un conjunto para almacenar los títulos y precios únicos
    productos_set = set()
    
    # Iterar sobre los elementos y obtener los títulos y precios
    for title, price in zip(titles, prices):
        # Obtener el título y el precio y agregarlos a la lista
        titulo = title.text.strip()
        precio = price.text.strip().replace('$', '').replace('.', '').replace(',', '.')  # Formatear el precio
        productos_set.add((titulo, float(precio)))

    
    # Cierra el navegador
    driver.quit()
    
    # Convertir el conjunto a una lista y ordenarla por título
    productos = [{'Título': titulo, 'Precio': precio} for titulo, precio in sorted(productos_set, key=lambda x: x[0])]
    
    # Calcular la similitud entre el nombre del producto consultado y los nombres de los productos encontrados
    productos_similares = []
    for producto in productos:
        similitud = fuzz.token_sort_ratio(nombre_producto.lower(), producto['Título'].lower())
        if similitud >= umbral_similitud:
            producto['Similitud'] = similitud
            productos_similares.append(producto)
    
    return productos_similares

In [1950]:
# Función para obtener la información de un producto en la página de Selma Digital
def obtener_info_selma_digital(nombre_producto, umbral_similitud=60):
    # URL de Selma Digital
    url_selma_digital = f'https://selmadigital.com/shop?search={nombre_producto}'

    # Realizar la solicitud HTTP
    response_selma_digital = requests.get(url_selma_digital)

    # Verificar si la solicitud fue exitosa (código de estado 200)
    if response_selma_digital.status_code == 200:
        # Parsear el contenido HTML con BeautifulSoup
        soup_selma_digital = BeautifulSoup(response_selma_digital.content, 'html.parser')

        # Buscar los elementos que contienen el título con la clase "product-card-design8-vertical__name"
        titles = soup_selma_digital.find_all('div', class_='product-card-design8-vertical__name')

        # Buscar los elementos que contienen el precio con la clase "product-card-design8-vertical__price"
        prices = soup_selma_digital.find_all('div', class_='product-card-design8-vertical__price')
        
        # Crear un conjunto para almacenar los títulos y precios únicos
        productos_set = set()

        # Iterar sobre los elementos y obtener los títulos y precios
        for title, price in zip(titles, prices):
            # Obtener el título y el precio y agregarlos a la lista
            titulo = title.text.strip()
            precio = price.text.strip().replace('$', '').replace('.', '').replace(',', '.')#price.text.strip().replace('$', '').replace(',', '')  # Eliminar caracteres no numéricos del precio
            productos_set.add((titulo, float(precio)))

        # Convertir el conjunto a una lista y ordenarla por título
        productos = [{'Título': titulo, 'Precio': precio} for titulo, precio in sorted(productos_set, key=lambda x: x[0])]
        
        # Calcular la similitud entre el nombre del producto consultado y los nombres de los productos encontrados
        productos_similares = []
        for producto in productos:
            similitud = fuzz.token_sort_ratio(nombre_producto.lower(), producto['Título'].lower())
            if similitud >= umbral_similitud:
                producto['Similitud'] = similitud
                productos_similares.append(producto)
        
        return productos_similares

    return None

In [1951]:
#funcion para consultar en la pagina de openfarma
def obtener_info_openfarma(nombre_producto, umbral_similitud=60):
    # URL de Openfarma
    url_openfarma = f'https://www.openfarma.com.ar/products?utf8=%E2%9C%93&keywords={nombre_producto}'

    # Realizar la solicitud HTTP
    response_openfarma = requests.get(url_openfarma)

    # Verificar si la solicitud fue exitosa (código de estado 200)
    if response_openfarma.status_code == 200:
        # Parsear el contenido HTML con BeautifulSoup
        soup_openfarma = BeautifulSoup(response_openfarma.content, 'html.parser')

        # Buscar los elementos que contienen el título con la clase "card-title"
        titles = soup_openfarma.find_all('h2', class_='card-title')

        # Buscar los elementos que contienen el precio con la clase "regular"
        prices_regular = soup_openfarma.find_all('span', class_='regular')
        
        # Buscar los elementos que contienen el precio con la clase "promo"
        prices_promo = soup_openfarma.find_all('span', class_='promo')
        
        prices = prices_regular
        
        if len(prices_promo) > 0:
            prices = prices_promo
    

         # Crear una lista para almacenar los productos
        productos_set = set()

        # Iterar sobre los elementos y obtener los títulos y precios
        for title, price in zip(titles, prices):
            # Obtener el título y el precio y agregarlos a la lista
            titulo = title.text.strip()
            precio = price.text.strip().replace('$', '').replace('.', '').replace(',', '.')  # Formatear el precio
            productos_set.add((titulo, float(precio)))

        # Convertir el conjunto a una lista y ordenarla por título
        productos = [{'Título': titulo, 'Precio': precio} for titulo, precio in sorted(productos_set, key=lambda x: x[0])]
        
        # Calcular la similitud entre el nombre del producto consultado y los nombres de los productos encontrados
        productos_similares = []
        for producto in productos:
            similitud = fuzz.token_sort_ratio(nombre_producto.lower(), producto['Título'].lower())
            if similitud >= umbral_similitud:
                producto['Similitud'] = similitud
                productos_similares.append(producto)
        
        return productos_similares

    return None

In [1952]:
# función para comparar precios consultados para selma digital y openfarma
def comparar_precios(productos_selma_digital, productos_openfarma):
    precios_selma_digital = {producto['Título']: producto['Precio'] for producto in productos_selma_digital}
    precios_openfarma = {producto['Título']: producto['Precio'] for producto in productos_openfarma}

    for titulo_selma, precio_selma in precios_selma_digital.items():
        for titulo_openfarma, precio_openfarma in precios_openfarma.items():
            similitud = fuzz.token_sort_ratio(titulo_selma, titulo_openfarma)
            if similitud >= 80:  # Umbral de similitud
                if precio_selma < precio_openfarma:
                    print(f'Selma Digital tiene un precio más bajo para "{titulo_selma}": ${precio_selma}')
                elif precio_selma > precio_openfarma:
                    print(f'Openfarma tiene un precio más bajo para "{titulo_openfarma}": ${precio_openfarma}')
                else:
                    print(f'Ambos tienen el mismo precio para "{titulo_selma}": ${precio_selma}')

In [1953]:
def comparar_precios_3_paginas(productos_selma_digital, productos_openfarma, productos_farmacity):
    precios_selma_digital = {producto['Título']: producto['Precio'] for producto in productos_selma_digital}
    precios_openfarma = {producto['Título']: producto['Precio'] for producto in productos_openfarma}
    precios_farmacity = {producto['Título']: producto['Precio'] for producto in productos_farmacity}

    for titulo_selma, precio_selma in precios_selma_digital.items():
        for titulo_openfarma, precio_openfarma in precios_openfarma.items():
            similitud = fuzz.token_sort_ratio(titulo_selma, titulo_openfarma)
            if similitud >= 80:  # Umbral de similitud
                if precio_selma < precio_openfarma:
                    print(f'Selma Digital tiene un precio más bajo para "{titulo_selma}": ${precio_selma}')
                elif precio_selma > precio_openfarma:
                    print(f'Openfarma tiene un precio más bajo para "{titulo_openfarma}": ${precio_openfarma}')
                else:
                    print(f'Ambos tienen el mismo precio para "{titulo_selma}": ${precio_selma}')
    
    for titulo_selma, precio_selma in precios_selma_digital.items():
        for titulo_farmacity, precio_farmacity in precios_farmacity.items():
            similitud = fuzz.token_sort_ratio(titulo_selma, titulo_farmacity)
            if similitud >= 80:  # Umbral de similitud
                if precio_selma < precio_farmacity:
                    print(f'Selma Digital tiene un precio más bajo para "{titulo_selma}": ${precio_selma}')
                elif precio_selma > precio_farmacity:
                    print(f'Farmacity tiene un precio más bajo para "{titulo_farmacity}": ${precio_farmacity}')
                else:
                    print(f'Ambos tienen el mismo precio para "{titulo_selma}": ${precio_selma}')
    
    for titulo_openfarma, precio_openfarma in precios_openfarma.items():
        for titulo_tercera, precio_tercera in precios_farmacity.items():
            similitud = fuzz.token_sort_ratio(titulo_openfarma, titulo_tercera)
            if similitud >= 80:  # Umbral de similitud
                if precio_openfarma < precio_tercera:
                    print(f'Openfarma tiene un precio más bajo para "{titulo_openfarma}": ${precio_openfarma}')
                elif precio_openfarma > precio_tercera:
                    print(f'Farmacity tiene un precio más bajo para "{titulo_farmacity}": ${precio_farmacity}')
                else:
                    print(f'Ambos tienen el mismo precio para "{titulo_openfarma}": ${precio_openfarma}')

In [1954]:
def comparar_precios_3_paginas_2(productos_selma_digital, productos_openfarma, productos_farmacity, umbral=80):
    precios_selma_digital = {producto['Título']: producto['Precio'] for producto in productos_selma_digital}
    precios_openfarma = {producto['Título']: producto['Precio'] for producto in productos_openfarma}
    precios_farmacity = {producto['Título']: producto['Precio'] for producto in productos_farmacity}

    for titulo_selma, precio_selma in precios_selma_digital.items():
        for titulo_openfarma, precio_openfarma in precios_openfarma.items():
            similitud = fuzz.token_sort_ratio(titulo_selma, titulo_openfarma)
            if similitud >= umbral:  # Umbral de similitud ajustable
                if precio_selma < precio_openfarma:
                    print(f'Selma Digital tiene un precio más bajo para "{titulo_selma}": ${precio_selma}')
                elif precio_selma > precio_openfarma:
                    print(f'Openfarma tiene un precio más bajo para "{titulo_openfarma}": ${precio_openfarma}')
                else:
                    print(f'Ambos tienen el mismo precio para "{titulo_selma}": ${precio_selma}')
    
    for titulo_selma, precio_selma in precios_selma_digital.items():
        for titulo_farmacity, precio_farmacity in precios_farmacity.items():
            similitud = fuzz.token_sort_ratio(titulo_selma, titulo_farmacity)
            if similitud >= umbral:  # Umbral de similitud ajustable
                if precio_selma < precio_farmacity:
                    print(f'Selma Digital tiene un precio más bajo para "{titulo_selma}": ${precio_selma}')
                elif precio_selma > precio_farmacity:
                    print(f'Farmacity tiene un precio más bajo para "{titulo_farmacity}": ${precio_farmacity}')
                else:
                    print(f'Ambos tienen el mismo precio para "{titulo_selma}": ${precio_selma}')
    
    for titulo_openfarma, precio_openfarma in precios_openfarma.items():
        for titulo_tercera, precio_tercera in precios_farmacity.items():
            similitud = fuzz.token_sort_ratio(titulo_openfarma, titulo_tercera)
            if similitud >= umbral:  # Umbral de similitud ajustable
                if precio_openfarma < precio_tercera:
                    print(f'Openfarma tiene un precio más bajo para "{titulo_openfarma}": ${precio_openfarma}')
                elif precio_openfarma > precio_tercera:
                    print(f'Farmacity tiene un precio más bajo para "{titulo_farmacity}": ${precio_farmacity}')
                else:
                    print(f'Ambos tienen el mismo precio para "{titulo_openfarma}": ${precio_openfarma}')

In [1955]:
def comparar_precios_3_paginas_3(productos_selma_digital, productos_openfarma, productos_farmacity, umbral=60):
    resultados = []  # Lista para almacenar los resultados de la comparación
    
    for producto_selma in productos_selma_digital:
        titulo_selma = producto_selma['Título']
        precio_selma = producto_selma['Precio']
        
        # Bandera para verificar si ya se ha agregado el producto de Selma Digital
        #selma_agregado = False
        
        for producto_openfarma in productos_openfarma:
            titulo_openfarma = producto_openfarma['Título']
            precio_openfarma = producto_openfarma['Precio']
            similitud_openfarma = fuzz.token_sort_ratio(titulo_selma, titulo_openfarma)
            
            if similitud_openfarma >= umbral:
                if precio_selma < precio_openfarma:
                    #if selma_agregado == False:
                    resultados.append(f'Selma Digital tiene un precio más bajo para "{titulo_selma}": ${precio_selma}')
                    #selma_agregado = True
                elif precio_selma > precio_openfarma:
                    resultados.append(f'Openfarma tiene un precio más bajo para "{titulo_openfarma}": ${precio_openfarma}')
                else:
                    resultados.append(f'Ambos tienen el mismo precio para "{titulo_selma}": ${precio_selma}')
        
        for producto_farmacity in productos_farmacity:
            titulo_farmacity = producto_farmacity['Título']
            precio_farmacity = producto_farmacity['Precio']
            similitud_farmacity = fuzz.token_sort_ratio(titulo_selma, titulo_farmacity)
            
            if similitud_farmacity >= umbral:
                
                if precio_selma < precio_farmacity:
                    #if selma_agregado == False:
                    resultados.append(f'Selma Digital tiene un precio más bajo para "{titulo_selma}": ${precio_selma}')
                    #selma_agregado = True
                elif precio_selma > precio_farmacity:
                    resultados.append(f'Farmacity tiene un precio más bajo para "{titulo_farmacity}": ${precio_farmacity}')
                else:
                    resultados.append(f'Ambos tienen el mismo precio para "{titulo_selma}": ${precio_selma}')
        
        
    return resultados

In [1956]:
def comparar_precios_3_paginas_4(productos_selma_digital, productos_openfarma, productos_farmacity, umbral=60):
    todos_productos = []

    # Agregar productos de Selma Digital
    for producto in productos_selma_digital:
        todos_productos.append({'pagina': 'Selma Digital', 'titulo': producto['Título'], 'precio': producto['Precio']})

    # Agregar productos de Openfarma
    for producto in productos_openfarma:
        todos_productos.append({'pagina': 'Openfarma', 'titulo': producto['Título'], 'precio': producto['Precio']})

    # Agregar productos de Farmacity
    for producto in productos_farmacity:
        todos_productos.append({'pagina': 'Farmacity', 'titulo': producto['Título'], 'precio': producto['Precio']})

    # Ordenar productos por precio de forma ascendente
    todos_productos.sort(key=itemgetter('precio'))

    # Tomar los primeros 10 productos
    top_10_productos = todos_productos[:10]

    # Crear la lista de resultados
    resultados = ["Comparación de precios entre las tres páginas (Selma Digital, Openfarma y Farmacity):"]
    
    for i, producto in enumerate(top_10_productos, start=1):
        resultados.append(f"{i}. {producto['pagina']} tiene el producto '{producto['titulo']}' a ${producto['precio']}")

    return resultados

In [1957]:
# Prueba de productos buscados
#'Crema De Manos Antimanchas Eucerin Anti Pigment X 75 Ml'
#'Crema de Manos Antimanchas Eucerin Anti-Pigment con Fps 30 x 75 ml'
#'Desodorante Antitranspirante'
#'jabon'
#'jabón'
#'tampon'

print('Ingrese el producto a consultar')

# Nombre del producto a buscar en ambas farmacias
nombre_producto = input()

print('Producto consultado: ',nombre_producto)

Ingrese el producto a consultar
Producto consultado:  Crema de Manos Antimanchas Eucerin


In [1958]:
# Obtener e imprimir la información de los productos de Selma Digital
productos_selma_digital = obtener_info_selma_digital(nombre_producto)
if productos_selma_digital:
    print(f'Productos encontrados en Selma Digital para "{nombre_producto}":')
    for idx, producto in enumerate(productos_selma_digital, start=1):
        print(f'#{idx} - Título: {producto["Título"]}, Precio: ${producto["Precio"]}')
else:
    print(f'No se pudo obtener la información de Selma Digital para "{nombre_producto}".')



Productos encontrados en Selma Digital para "Crema de Manos Antimanchas Eucerin":
#1 - Título: Crema de manos antimanchas Eucerin ANTI-PIGMENT con FPS 30 x 75 ml, Precio: $20261.0


In [1959]:
# Obtener e imprimir la información de los productos de Openfarma
productos_openfarma = obtener_info_openfarma(nombre_producto)
if productos_openfarma:
    print(f'\nProductos encontrados en Openfarma para "{nombre_producto}":')
    for idx, producto in enumerate(productos_openfarma, start=1):
        print(f'#{idx} - Título: {producto["Título"]}, Precio: ${producto["Precio"]}')
else:
    print(f'No se pudo obtener la información de Openfarma para "{nombre_producto}".')



Productos encontrados en Openfarma para "Crema de Manos Antimanchas Eucerin":
#1 - Título: Crema De DÃ­a Eucerin Anti Pigment Fps 30 X 50ml, Precio: $7454.3
#2 - Título: Crema De DÃ­a Eucerin Anti Pigment Fps 30 X 50ml, Precio: $21103.2
#3 - Título: Crema De Manos Antimanchas Eucerin Anti Pigment X 75 Ml, Precio: $4674.6
#4 - Título: Crema De Manos Antimanchas Eucerin Anti Pigment X 75 Ml, Precio: $20267.0


In [1960]:
# Obtener e imprimir la información de los productos de Farmacity
productos_farmacity = obtener_info_farmacity(nombre_producto)
if productos_farmacity:
    print(f'\nProductos encontrados en Farmacity para "{nombre_producto}":')
    for idx, producto in enumerate(productos_farmacity, start=1):
        print(f'#{idx} - Título: {producto["Título"]}, Precio: ${producto["Precio"]}')
else:
    print(f'No se pudo obtener la información de Farmacity para "{nombre_producto}".')


Productos encontrados en Farmacity para "Crema de Manos Antimanchas Eucerin":
#1 - Título: Crema de Manos Antimanchas Eucerin Anti-Pigment con Fps 30 x 75 ml, Precio: $21826.0


In [1961]:
if productos_selma_digital or productos_openfarma or productos_farmacity:
    pagina_mas_economica_selma = None
    pagina_mas_economica_openfarma = None
    pagina_mas_economica_farmacity = None
    productos_mas_economicos_selma = None
    productos_mas_economicos_openfarma = None
    productos_mas_economicos_farmacity = None
    pagina_mas_economica_selma = "Selma Digital"
    pagina_mas_economica_openfarma = "Openfarma"
    pagina_mas_economica_farmacity = "Farmacity"

    if productos_selma_digital:
        #print("Los productos fueron encontrados en Selma Digital.")
        pagina_mas_economica_selma = "Selma Digital"
        productos_mas_economicos_selma = productos_selma_digital
    if productos_openfarma:
        #print("Los productos fueron encontrados en Openfarma.")
        pagina_mas_economica_openfarma = "Openfarma"
        productos_mas_economicos_openfarma = productos_openfarma
    if productos_farmacity:
        #print("Los productos fueron encontrados en Farmacity.")
        pagina_mas_economica_farmacity = "Farmacity"
        productos_mas_economicos_farmacity = productos_farmacity

    producto_mas_economico_selma = min(productos_mas_economicos_selma, key=lambda x: x['Precio']) if productos_mas_economicos_selma else None
    producto_mas_economico_openfarma = min(productos_mas_economicos_openfarma, key=lambda x: x['Precio']) if productos_mas_economicos_openfarma else None
    producto_mas_economico_farmacity = min(productos_mas_economicos_farmacity, key=lambda x: x['Precio']) if productos_mas_economicos_farmacity else None
    
    resultados = comparar_precios_3_paginas_4(productos_selma_digital, productos_openfarma, productos_farmacity)
    
    print("\n")
    
    if resultados:
        #print("Comparación de precios entre las tres páginas (Selma Digital, Openfarma y Farmacity):\n")
        for resultado in resultados:            
            print(resultado)
    else:
        print("No se encontraron resultados para comparar precios entre las tres páginas.\n")
    
    if producto_mas_economico_selma:
        print("\n")
        print("Los productos fueron encontrados en Selma Digital.")
        print(f"El producto más económico encontrado en {pagina_mas_economica_selma} es:")
        print("Título:", producto_mas_economico_selma['Título'])
        print("Precio:", producto_mas_economico_selma['Precio'])
        print()
    else:
        print(f"No se encontraron productos en {pagina_mas_economica_selma}.\n")
        
    if producto_mas_economico_openfarma:
        print("Los productos fueron encontrados en Openfarma.")
        print(f"El producto más económico encontrado en {pagina_mas_economica_openfarma} es:")
        print("Título:", producto_mas_economico_openfarma['Título'])
        print("Precio:", producto_mas_economico_openfarma['Precio'])
        print()
    else:
        print(f"No se encontraron productos en {pagina_mas_economica_openfarma}.\n")
        
    if producto_mas_economico_farmacity:
        print("Los productos fueron encontrados en Farmacity.")
        print(f"El producto más económico encontrado en {pagina_mas_economica_farmacity} es:")
        print("Título:", producto_mas_economico_farmacity['Título'])
        print("Precio:", producto_mas_economico_farmacity['Precio'])
        print()
    else:
        print(f"No se encontraron productos en {pagina_mas_economica_farmacity}.\n")
else:
    print("No se encontraron productos en ninguna página.")



Comparación de precios entre las tres páginas (Selma Digital, Openfarma y Farmacity):
1. Openfarma tiene el producto 'Crema De Manos Antimanchas Eucerin Anti Pigment X 75 Ml' a $4674.6
2. Openfarma tiene el producto 'Crema De DÃ­a Eucerin Anti Pigment Fps 30 X 50ml' a $7454.3
3. Selma Digital tiene el producto 'Crema de manos antimanchas Eucerin ANTI-PIGMENT con FPS 30 x 75 ml' a $20261.0
4. Openfarma tiene el producto 'Crema De Manos Antimanchas Eucerin Anti Pigment X 75 Ml' a $20267.0
5. Openfarma tiene el producto 'Crema De DÃ­a Eucerin Anti Pigment Fps 30 X 50ml' a $21103.2
6. Farmacity tiene el producto 'Crema de Manos Antimanchas Eucerin Anti-Pigment con Fps 30 x 75 ml' a $21826.0


Los productos fueron encontrados en Selma Digital.
El producto más económico encontrado en Selma Digital es:
Título: Crema de manos antimanchas Eucerin ANTI-PIGMENT con FPS 30 x 75 ml
Precio: 20261.0

Los productos fueron encontrados en Openfarma.
El producto más económico encontrado en Openfarma es